## Microtask0

### Problem Statement:
Use this notebook implementing the Code_Changes metric (see it in MyBinder) as an example of how to collect the data, producing a single JSON file per data source, with all items (commits, issues, pull/merge requests) in it. Produce one notebook per data source (git, GitHub/GitLab issues, GitHub pull requests / GitLab merge requests) showing a summary of the contents of that file (number of items in it, and number of different identities in it counting authors/committers for git, submitters for issues and pull/merge requests). This microtask is mandatory, to show that you can retrieve data and produde a notebook showing it. In each notebook, include also the list of repositories retrieved, and the date of retrieval, using data available in the JSON file.

### Project: progit

### Repositories

In [1]:
import json
import datetime
from collections import defaultdict
from pprint import pprint

### Getting the data

In [2]:
github_url = "https://github.com/"
owner = "progit"
repos_used = ["progit2-ru", "progit2-zh"]
# github_url = "https://github.com/"
# owner = "django"
# repos_used = ["djangoproject.com"]
repo_urls = [github_url + owner + "/" + repo_used for repo_used in repos_used]
auth_token = "f4d98f2cef0a0b4873b723fe3ee4574765e8b37f"
print(repo_urls)



['https://github.com/progit/progit2-ru', 'https://github.com/progit/progit2-zh']


In [3]:
for repo, repo_url in zip(repos_used, repo_urls):
    print(repo, repo_url)
    if repo == repos_used[0]:
        !perceval git --json-line $repo_url > progit.json
        
    else:
        !perceval git --json-line $repo_url >> progit.json
    print("...")
    !perceval github -t $auth_token --json-line --sleep-for-rate --category pull_request $owner $repo >> progit.json
    print("...")
    !perceval github -t $auth_token --json-line --sleep-for-rate --category issue $owner $repo >> progit.json
    print("...")

progit2-ru https://github.com/progit/progit2-ru
[2019-03-20 17:08:20,163] - Sir Perceval is on his quest.
[2019-03-20 17:09:59,575] - Fetching commits: 'https://github.com/progit/progit2-ru' git repository from 1970-01-01 00:00:00+00:00 to 2100-01-01 00:00:00+00:00; all branches
[2019-03-20 17:10:13,041] - Fetch process completed: 1292 commits fetched
[2019-03-20 17:10:13,041] - Sir Perceval completed his quest.
...
[2019-03-20 17:10:13,550] - Sir Perceval is on his quest.
[2019-03-20 17:10:19,537] - Getting info for https://api.github.com/users/Bakuutin
[2019-03-20 17:10:20,552] - Getting info for https://api.github.com/users/sadfuzzy
[2019-03-20 17:10:30,354] - Getting info for https://api.github.com/users/madhead
[2019-03-20 17:10:35,711] - Getting info for https://api.github.com/users/dmitry-krasilnikov
[2019-03-20 17:10:39,989] - Getting info for https://api.github.com/users/lyobzik
[2019-03-20 17:10:46,027] - Getting info for https://api.github.com/users/evgkrsk
[2019-03-20 17:10

[2019-03-20 17:20:10,998] - Getting info for https://api.github.com/users/vmesh
[2019-03-20 17:20:14,681] - Getting info for https://api.github.com/users/sosnovskyas
[2019-03-20 17:20:16,121] - Getting info for https://api.github.com/users/mfilippov
[2019-03-20 17:20:19,192] - Getting info for https://api.github.com/users/ssarkisy
[2019-03-20 17:20:21,236] - Getting info for https://api.github.com/users/Dehax
[2019-03-20 17:20:24,438] - Getting info for https://api.github.com/users/igrik512
[2019-03-20 17:20:26,566] - Getting info for https://api.github.com/users/Snowbridge
[2019-03-20 17:20:28,405] - Getting info for https://api.github.com/users/v-oz
[2019-03-20 17:20:32,306] - Getting info for https://api.github.com/users/proninyaroslav
[2019-03-20 17:20:34,462] - Getting info for https://api.github.com/users/7rulnik
[2019-03-20 17:20:35,907] - Getting info for https://api.github.com/users/lastwhitedance
[2019-03-20 17:20:37,213] - Getting info for https://api.github.com/users/Rustam

[2019-03-20 17:28:29,115] - Getting info for https://api.github.com/users/peizh
[2019-03-20 17:28:30,092] - Getting info for https://api.github.com/users/byr-gdp
[2019-03-20 17:28:33,168] - Getting info for https://api.github.com/users/wogong
[2019-03-20 17:28:39,528] - Getting info for https://api.github.com/users/buginux
[2019-03-20 17:28:56,604] - Getting info for https://api.github.com/users/morefreeze
[2019-03-20 17:29:01,236] - Getting info for https://api.github.com/users/calidion
[2019-03-20 17:29:39,943] - Getting info for https://api.github.com/users/ZKHelloworld
[2019-03-20 17:29:44,448] - Getting info for https://api.github.com/users/secondwtq
[2019-03-20 17:30:08,670] - Getting info for https://api.github.com/users/leshiv
[2019-03-20 17:30:12,779] - Getting info for https://api.github.com/users/summershrimp
[2019-03-20 17:30:23,750] - Getting info for https://api.github.com/users/tvvocold
[2019-03-20 17:30:28,284] - Getting info for https://api.github.com/users/Kyle-ak
[20

[2019-03-20 17:38:09,976] - Getting info for https://api.github.com/users/xinqiu
[2019-03-20 17:38:18,500] - Getting info for https://api.github.com/users/IceNature
[2019-03-20 17:38:20,169] - Getting info for https://api.github.com/users/wogong
[2019-03-20 17:38:23,818] - Getting info for https://api.github.com/users/buginux
[2019-03-20 17:38:27,299] - Getting info for https://api.github.com/users/pktangyue
[2019-03-20 17:38:31,598] - Getting info for https://api.github.com/users/jiangxin
[2019-03-20 17:38:34,468] - Getting info for https://api.github.com/users/morefreeze
[2019-03-20 17:38:36,631] - Getting info for https://api.github.com/users/calidion
[2019-03-20 17:38:39,367] - Getting info for https://api.github.com/users/ahlijin
[2019-03-20 17:38:40,472] - Getting info for https://api.github.com/users/jiayouxjh
[2019-03-20 17:38:46,813] - Getting info for https://api.github.com/users/macrov
[2019-03-20 17:38:54,350] - Getting info for https://api.github.com/users/Kyle-ak
[2019-03

[2019-03-20 17:43:12,625] - Getting info for https://api.github.com/users/wsxyeah
[2019-03-20 17:43:17,899] - Getting info for https://api.github.com/users/sooYo
[2019-03-20 17:43:19,147] - Getting info for https://api.github.com/users/fanofxiaofeng
[2019-03-20 17:43:21,392] - Getting info for https://api.github.com/users/perry2008084
[2019-03-20 17:43:22,831] - Getting info for https://api.github.com/users/shukebeta
[2019-03-20 17:43:24,811] - Getting info for https://api.github.com/users/wangfpp
[2019-03-20 17:43:25,909] - Getting info for https://api.github.com/users/zhangnew
[2019-03-20 17:43:28,770] - Getting info for https://api.github.com/users/wqfeng
[2019-03-20 17:43:30,201] - Getting info for https://api.github.com/users/shangleixun
[2019-03-20 17:43:31,832] - Getting info for https://api.github.com/users/heguangzhi
[2019-03-20 17:43:34,093] - Getting info for https://api.github.com/users/yssgithub
[2019-03-20 17:43:35,906] - Getting info for https://api.github.com/users/jose

In [4]:
class Code_Changes:
               
    
    def __init__(self, path_to_file):
        
        self.clean_data = defaultdict(list)
        
        with open(path_to_file, 'r') as raw_data:
            for line in raw_data:
                line = json.loads(line)

                clean_line = dict()
                if line['category'] == "commit":
                    clean_line = self._clean_commit(line)
                    
                elif line['category'] == "issue":
                    clean_line = self._clean_issue(line)

                elif line['category'] == "pull_request":
                    clean_line = self._clean_pr(line)

                self.clean_data[line['category']].append(clean_line)        
    
    
    def number_of_repos(self):
        return len(repos_used)
    
    def total_commits(self):
        return len(self.clean_data['commit'])
    
    def total_commits_per_repo(self):
        commits_per_repo = {el:0 for el in repo_urls}
        
        for commit in self.clean_data['commit']:
            
            commits_per_repo[commit['repo']] += 1
    
        return commits_per_repo
    
    def count_from_to(self, start=None, end=None, type_of_date="author_date", empty=True, merge=True):
        # commit_list has elements of a specific category
        category = "commit"
        commit_list = self.clean_data[category]
        start_date = datetime.datetime.strptime(start, "%Y-%m-%d") if start is not None else datetime.datetime.min
        end_date = datetime.datetime.strptime(end, "%Y-%m-%d") if end is not None else datetime.datetime.max
        
        required_commit_set = set()
        for elem in commit_list:
            if start_date <= self._clean_date(elem[type_of_date]) <= end_date:
                if (empty) or (not empty and elem['files_action'] != 0):
                    if (merge) or (not merge and elem['merge'] == False):

                        required_commit_set.add(elem['hash'])
        return len(required_commit_set)
                    
    
    # private methods to clean data ---------------------------
    
    @staticmethod
    def _clean_date(date_long_format):
        datetimeobj = datetime.datetime.strptime(date_long_format, "%a %b %d %H:%M:%S %Y %z")
        datetimeobj = datetimeobj.replace(tzinfo=None)
    
        return datetimeobj
    
    @staticmethod                
    def _clean_commit(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line = {
            'repo': repo_name,
            'hash': line_data['commit'],
            'category': "commit",
            'commit': line_data['Commit'],
            'author': line_data['Author'],
            'author_date': line_data['AuthorDate'],
            'commit_date': line_data['CommitDate'],
            'files_no': len(line_data['files'])
        }
        
        actions = 0
        
        cleaned_line['files_action'] = actions
        cleaned_line['merge'] = 'Merge' in line_data
        
        for file in line_data['files']:
            if 'action' in file:
                actions += 1
                cleaned_line['files_action'] = actions
                cleaned_line['merge'] = 'Merge' in line_data
        return cleaned_line
    
    @staticmethod
    def _clean_issue(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line ={
            'repo': repo_name,
            'hash': line_data['id'],
            'category': "issue",
            'author': line_data['user']['login'],
            'created_date': line_data['created_at'],
            'current_status': line_data['state']   
        }
        
        return cleaned_line
    
    @staticmethod
    def _clean_pr(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line ={
            'repo': repo_name,
            'hash': line_data['id'],
            'category': "pull_request",
            'author': line_data['user']['login'],
            'created_date': line_data['created_at'],
            'current_status': line_data['state']   
        }
        
        return cleaned_line
    

# Analyzing the data using the Class

In [7]:
commits_data = Code_Changes('./progit.json')

## Total number of commits 

In [8]:
print("The total number of commits in all repos is: ", commits_data.total_commits())
print("The number of commits repo-wise is ", commits_data.total_commits_per_repo())

The total number of commits in all repos is:  3024
The number of commits repo-wise is  {'https://github.com/progit/progit2-ru': 1292, 'https://github.com/progit/progit2-zh': 1732}


## Total number of commits between dates

In [9]:
print("Code changes count all period:", commits_data.count_from_to())
print("Code changes count from 2018-01-01 to 2018-07-01:",
      commits_data.count_from_to(start="2018-01-01", end="2019-07-01"))
print("Code changes count from 2018-01-01 to 2019-07-01 (no merge commits):",
      commits_data.count_from_to(start="2018-01-01", end="2018-07-01", merge=False))


Code changes count all period: 2402
Code changes count from 2018-01-01 to 2018-07-01: 77


KeyError: 'merge'

# Analyzing the json file directly

In [ ]:
github_data = defaultdict(list)

count = 0
with open('trial.json', 'r') as github_data_file:
    for line in github_data_file:
        data_line = json.loads(line)
        count += 1

        category = data_line['category']
        data = data_line['data']
        github_data[category].append(data)
            
print(len(github_data['commit']))

## Total number of commits in the master branch


In [ ]:
master_commits = set()

for elem in github_data['commit']:
    if 'HEAD -> refs/heads/master' in elem['refs']:
        master_commits.add(elem['commit'])       
        for parent in elem['parents']:
            
            if parent not in master_commits:
                master_commits.add(parent)
                
print(len(master_commits))
        

## Total number of non empty commits

In [ ]:
num_empty_commits = 0

for commit in github_data['commits']:
    for file in commit['files']:
        if 'action' in file:
            num_empty_commits += 1
            break
            
print(num_empty_commits)
            

## Total number of non - merge commits

In [ ]:
count = 0

for commit in github_data['commit']:
    if 'Merge' not in commit:
        count += 1
        
print("Number of non-merge commits is: %d" %count)

# Pull Requests and Issues

## Total number of pull requests and issues

In [ ]:
total_issues = len(github_data["issue"])

print("The number of issues is {0}".format(total_issues))
print("The number of pull requests is {0}".format(len(github_data["pull_request"])))

## Total number of open and closed issues

In [ ]:
num_open_issues = 0
for issue in github_data["issue"]:
    if issue['state'] == "open":
        num_open_issues += 1
        
print("The number of open issues is ", num_open_issues)
print("The number of closed issues is ", total_issues - num_open_issues)